In [12]:
import json

jsonl_path = "/mnt/fast10/brandon/mmr_rollout_data/final_combined_MC_and_verification_files/CLEVR_final_mc_rollouts_with_all_models_verification_merged.jsonl"
target_rollout_uuid = "8f767ba9-906c-45fb-9a18-1ca1b7758643"
target_field = "o4_mini_isVerified"
target_value = None

# Load all lines
with open(jsonl_path, "r") as f:
    lines = f.readlines()

data = [json.loads(line) for line in lines]

# Find and update the target row
matches = [i for i, row in enumerate(data) if row.get("rollout_uuid") == target_rollout_uuid]
if len(matches) != 1:
    raise ValueError(f"Expected exactly one match for rollout_uuid={target_rollout_uuid}, found {len(matches)}")

idx = matches[0]
print(f"updating {target_field} for target uuid {target_rollout_uuid}, from {data[idx][target_field]} to {target_value}")
print()
data[idx][target_field] = target_value

# before saving, load the array into a pandas dataframe and print the updated value where the target uuid is
import pandas as pd
df = pd.DataFrame(data)
print()
print(f"dataframe value of {target_field} for target uuid {target_rollout_uuid} is now updated to:")
print("-------------START-------------------")
print(df[df['rollout_uuid'] == target_rollout_uuid][target_field].values[0])
print("---------------END-----------------")

print("-------------START-------------------")
print(df[df['rollout_uuid'] == target_rollout_uuid].o4_mini_verification_solution.values[0])
print("---------------END-----------------")

# Save back to JSONL (overwrite)
with open(jsonl_path, "w") as f:
    for row in data:
        f.write(json.dumps(row, ensure_ascii=False) + "\n")

updating o4_mini_isVerified for target uuid 8f767ba9-906c-45fb-9a18-1ca1b7758643, from False to None


dataframe value of o4_mini_isVerified for target uuid 8f767ba9-906c-45fb-9a18-1ca1b7758643 is now updated to:
-------------START-------------------
None
---------------END-----------------
-------------START-------------------
<analysis_1>
Step 1 correctly notes that the scene contains only spheres and one cylinder, with no cubes present. This matches the image.
</analysis_1>

<analysis_2>
Step 2 correctly identifies metallic objects as the shiny, reflective gold spheres and classifies the matte-coloured spheres and the blue cylinder as non-metal. This matches the visual materials.
</analysis_2>

<analysis_3>
Step 3 claims there are two large gold (metallic) spheres, but the image shows only one truly large reflective gold sphere (front left). The only other reflective gold sphere (middle‐right) is much smaller in diameter. Thus there is only one large metal object, not two. This misc

In [13]:
# Reload the file and verify the update
with open(jsonl_path, "r") as f:
    updated_data = [json.loads(line) for line in f]

updated_row = next((row for row in updated_data if row.get("rollout_uuid") == target_rollout_uuid), None)
if updated_row is None:
    raise ValueError(f"Could not find row with rollout_uuid={target_rollout_uuid} after update")

print(f"After update, {target_field} for {target_rollout_uuid} is:", updated_row[target_field])

After update, o4_mini_isVerified for 8f767ba9-906c-45fb-9a18-1ca1b7758643 is: None


In [16]:
import os

data_dir = "/mnt/fast10/brandon/mmr_rollout_data/final_combined_MC_and_verification_files"

for file in os.listdir(data_dir):
    if file.endswith(".jsonl"):
        print(file)
        # break


# load the file into a pandas dataframe
# df = pd.read_json(file, lines=True)

CLEVR_final_mc_rollouts_with_all_models_verification_merged.jsonl
RAVEN_final_mc_rollouts_with_all_models_verification_merged.jsonl
dvqa_final_mc_rollouts_with_all_models_verification_merged.jsonl
AI2D_final_mc_rollouts_with_all_models_verification_merged.jsonl


In [ ]:
import json
import os
import re

# Directory path
data_dir = "/mnt/fast10/brandon/mmr_rollout_data/final_combined_MC_and_verification_files"

dataset_files = ["CLEVR_final_mc_rollouts_with_all_models_verification_merged.jsonl",
"RAVEN_final_mc_rollouts_with_all_models_verification_merged.jsonl",
"dvqa_final_mc_rollouts_with_all_models_verification_merged.jsonl",
"AI2D_final_mc_rollouts_with_all_models_verification_merged.jsonl"
]

# Array to store problematic rollout_uuids

# Loop through all JSONL files in the directory
for filename in dataset_files:
    problematic_uuids = []
    print(f"\nProcessing file: {filename}")
    filepath = os.path.join(data_dir, filename)
    
    # Load the JSONL file
    with open(filepath, 'r') as f:
        lines = f.readlines()
    
    data = [json.loads(line) for line in lines]
    
    # Check each item in the file
    for item in data:
        # Check if o4_mini_isVerified is not None
        if item.get('o4_mini_isVerified') == False:
            rollout_uuid = item.get('rollout_uuid')
            o4_mini_isVerified = item.get('o4_mini_isVerified')
            verification_solution = item.get('o4_mini_verification_solution', '')
            
            # Check if verification_solution contains required section headers
            has_visual_or_perception = (verification_solution.startswith('[Visual Elements]') or verification_solution.startswith('[Perception]'))
            
            # If missing required sections, record it
            if not (has_visual_or_perception):
                print(f"Missing required sections for rollout_uuid: {rollout_uuid}")
                print(f"o4_mini_isVerified: {o4_mini_isVerified}")
                print(f"Has Visual Elements or Perception header: {has_visual_or_perception}")
                print(f"Verification solution: {verification_solution}")
                
                problematic_uuids.append(rollout_uuid)

    print(f"\n\nSUMMARY:")
    print(f"Total problematic rollout_uuids found: {len(problematic_uuids)}")
    print(f"Problematic rollout_uuids: {problematic_uuids}")


Processing file: CLEVR_final_mc_rollouts_with_all_models_verification_merged.jsonl
Missing required sections for rollout_uuid: dd305859-b6af-4974-b08c-e6123f1c7faf
o4_mini_isVerified: False
Has Visual Elements or Perception header: False
Verification solution: <analysis_1>
The solution’s first perception step claims there is only one cylinder (the shiny blue one), but the image actually contains two cylinders: the large blue cylinder in front and a smaller cyan cylinder behind it. This miscount of cylinders undermines all subsequent reasoning.
</analysis_1>

<conclusion>
Incorrect
</conclusion>
Missing required sections for rollout_uuid: 204cc64c-2904-4ae6-b9c0-0cbffc10de0c
o4_mini_isVerified: False
Has Visual Elements or Perception header: False
Verification solution: <analysis_1>
The solution’s first perceptual step asserts there are three cubes (two purple and one blue). In the image, however, only two cubes are present: one purple cube on the left and one reflective blue cube on t

In [ ]:
import json
import os
import re

# Directory path
data_dir = "/mnt/fast10/brandon/mmr_rollout_data/final_combined_MC_and_verification_files_updated"

dataset_files = ["CLEVR_final_mc_rollouts_with_all_models_verification_merged.jsonl",
"RAVEN_final_mc_rollouts_with_all_models_verification_merged.jsonl",
"dvqa_final_mc_rollouts_with_all_models_verification_merged.jsonl",
"AI2D_final_mc_rollouts_with_all_models_verification_merged.jsonl"
]

# Array to store problematic rollout_uuids

# Loop through all JSONL files in the directory
for filename in dataset_files:
    problematic_uuids = []
    print(f"\nProcessing file: {filename}")
    filepath = os.path.join(data_dir, filename)
    
    # Load the JSONL file
    with open(filepath, 'r') as f:
        lines = f.readlines()
    
    data = [json.loads(line) for line in lines]
    
    # Check each item in the file
    for item in data:
        # Check if o4_mini_isVerified is not None
        if item.get('o4_mini_isVerified') == False:
            rollout_uuid = item.get('rollout_uuid')
            o4_mini_isVerified = item.get('o4_mini_isVerified')
            verification_solution = item.get('o4_mini_verification_solution', '')
            
            # Check if verification_solution contains required section headers
            has_visual_or_perception = (verification_solution.startswith('[Visual Elements]') or verification_solution.startswith('[Perception]'))

            has_valid_verification_solution = check_valid_verification_solution(verification_solution)

            # If missing required sections, record it
            if not (has_visual_or_perception):
                print(f"Missing required sections for rollout_uuid: {rollout_uuid}")
                print(f"o4_mini_isVerified: {o4_mini_isVerified}")
                print(f"Has Visual Elements or Perception header: {has_visual_or_perception}")
                print(f"Verification solution: {verification_solution}")
                
                problematic_uuids.append(rollout_uuid)

    print(f"\n\nSUMMARY:")
    print(f"Total problematic rollout_uuids found: {len(problematic_uuids)}")
    print(f"Problematic rollout_uuids: {problematic_uuids}")


Processing file: CLEVR_final_mc_rollouts_with_all_models_verification_merged.jsonl


SUMMARY:
Total problematic rollout_uuids found: 0
Problematic rollout_uuids: []

Processing file: RAVEN_final_mc_rollouts_with_all_models_verification_merged.jsonl


SUMMARY:
Total problematic rollout_uuids found: 0
Problematic rollout_uuids: []

Processing file: dvqa_final_mc_rollouts_with_all_models_verification_merged.jsonl


SUMMARY:
Total problematic rollout_uuids found: 0
Problematic rollout_uuids: []

Processing file: AI2D_final_mc_rollouts_with_all_models_verification_merged.jsonl


SUMMARY:
Total problematic rollout_uuids found: 0
Problematic rollout_uuids: []


# Verify In-place Edits add Up

In [6]:
import os
import json
from collections import Counter

dir_path = "/mnt/fast10/brandon/mmr_rollout_data/final_combined_MC_and_verification_files"
overall_counter = Counter()

for fname in os.listdir(dir_path):
    if fname.endswith(".jsonl"):
        fpath = os.path.join(dir_path, fname)
        counter = Counter()
        with open(fpath, "r") as f:
            for line in f:
                obj = json.loads(line)
                val = obj.get("o4_mini_isVerified", None)
                counter[val] += 1
                overall_counter[val] += 1
        print(f"{fname}: {dict(counter)}")

print(f"Total: {dict(overall_counter)}")

vqav2_final_mc_rollouts_with_all_models_verification_merged.jsonl: {False: 8684, True: 15241, None: 22}
InfoVQA_final_mc_rollouts_with_all_models_verification_merged.jsonl: {True: 23875, False: 4315, None: 19}
CLEVR_final_mc_rollouts_with_all_models_verification_merged.jsonl: {True: 39037, False: 20949, None: 6}
RAVEN_final_mc_rollouts_with_all_models_verification_merged.jsonl: {False: 265907, True: 6726, None: 13370}
dvqa_final_mc_rollouts_with_all_models_verification_merged.jsonl: {True: 68685, False: 21266, None: 48}
AI2D_final_mc_rollouts_with_all_models_verification_merged.jsonl: {False: 7368, True: 17225, None: 964}
Total: {False: 328489, True: 170789, None: 14429}


In [5]:
import os
import json
from collections import Counter

dir_path = "/mnt/fast10/brandon/mmr_rollout_data/final_combined_MC_and_verification_files_updated"
overall_counter = Counter()

for fname in os.listdir(dir_path):
    if fname.endswith(".jsonl"):
        fpath = os.path.join(dir_path, fname)
        counter = Counter()
        with open(fpath, "r") as f:
            for line in f:
                obj = json.loads(line)
                val = obj.get("o4_mini_isVerified", None)
                counter[val] += 1
                overall_counter[val] += 1
        print(f"{fname}: {dict(counter)}")

print(f"Total: {dict(overall_counter)}")

RAVEN_final_mc_rollouts_with_all_models_verification_merged.jsonl: {False: 265645, True: 6726, None: 13632}
dvqa_final_mc_rollouts_with_all_models_verification_merged.jsonl: {True: 68685, False: 21207, None: 107}
AI2D_final_mc_rollouts_with_all_models_verification_merged.jsonl: {False: 7366, True: 17225, None: 966}
Total: {False: 294218, True: 92636, None: 14705}


In [7]:
import os
import json
from collections import Counter

dir_path = "/mnt/fast10/brandon/mmr_rollout_data/completed_prm_training_data_conversion"
overall_counter = Counter()

for fname in os.listdir(dir_path):
    if fname.endswith(".jsonl"):
        fpath = os.path.join(dir_path, fname)
        counter = Counter()
        with open(fpath, "r") as f:
            for line in f:
                obj = json.loads(line)
                val = obj.get("o4_mini_isVerified", None)
                counter[val] += 1
                overall_counter[val] += 1
        print(f"{fname}: {dict(counter)}")

print(f"Total: {dict(overall_counter)}")

vqav2_final_mc_rollouts_with_all_models_verification_merged.jsonl: {False: 8684, True: 15241, None: 22}
InfoVQA_final_mc_rollouts_with_all_models_verification_merged.jsonl: {True: 23875, False: 4315, None: 19}
CLEVR_final_mc_rollouts_with_all_models_verification_merged.jsonl: {True: 39037, False: 20744, None: 211}
Total: {False: 33743, True: 78153, None: 252}
